# Proof of concept
## Get Groundstate of H2 with AdaptVQE just estimator, noisy backend simulation, noisy backend simulation with zne
- taken from: https://qiskit-community.github.io/qiskit-nature/howtos/adapt_vqe.html
- https://quantumcomputing.stackexchange.com/questions/34669/qiskit-noisy-fakemanila-simulator-zne
- https://docs.quantum.ibm.com/migration-guides/v2-primitives
- https://github.com/unitaryfund/research/blob/main/nepec/mitiq-qiskit-pec-nepec/mitiq-qiskit-zne-pec-nepec.ipynb

In [1]:
# imports
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
import numpy as np
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator

from qiskit_algorithms import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

import time
from datetime import datetime

## Define problem with ansatz

In [2]:
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto-3g")
problem = driver.run()

In [3]:
mapper = JordanWignerMapper()

In [4]:
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

In [5]:
# define vqe
ideal_vqe = VQE(Estimator(), ansatz, SLSQP())
ideal_vqe.initial_point = np.zeros(ansatz.num_parameters)

## now setup AdaptVQE with simulation

In [6]:
ideal_adapt_vqe = AdaptVQE(ideal_vqe)
ideal_adapt_vqe.supports_aux_operators = lambda: True  # temporary fix

## Now get some results AdaptVQE simulation

In [7]:
solver = GroundStateEigensolver(mapper, ideal_adapt_vqe)

In [8]:
start = time.time()
result = solver.solve(problem)
end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)
print(f"Total ground state energy = {result.total_energies[0]:.4f}")
print(f"Total number of iterations = {result.raw_result.num_iterations}")

Code execution time [sec]: 0.8029932975769043
Total ground state energy = -1.1373
Total number of iterations = 2


## now setup AdaptVQE with noisy backend

In [9]:
from qiskit_ibm_runtime.fake_provider import FakeVigo
device_backend = FakeVigo()

/tmp/ipykernel_12485/2647586769.py:2: DeprecationWarning: V1 fake backends are deprecated as of qiskit-ibm-runtime 0.24 and will be removed no sooner than 3 months after the release date. Please use V2 fake backends instead.
  device_backend = FakeVigo()


In [10]:
from qiskit_ibm_runtime import Estimator, Options

estimator = Estimator(device_backend)

/tmp/ipykernel_12485/105738346.py:3: DeprecationWarning: The Sampler and Estimator V1 primitives have been deprecated as of qiskit-ibm-runtime 0.23.0 and will be removed no sooner than 3 months after the release date. Please use the V2 Primitives. See the `V2 migration guide <https://docs.quantum.ibm.com/api/migration-guides/v2-primitives>`_. for more details
  estimator = Estimator(device_backend)


In [11]:
# define vqe
noisy_vqe = VQE(estimator, ansatz, SLSQP())
noisy_vqe.initial_point = np.zeros(ansatz.num_parameters)

In [12]:
noisy_adapt_vqe = AdaptVQE(noisy_vqe)
noisy_adapt_vqe.supports_aux_operators = lambda: True  # temporary fix

## Now get some results AdaptVQE noisy

In [13]:
solver = GroundStateEigensolver(mapper, noisy_adapt_vqe)

In [14]:
start = time.time()
result = solver.solve(problem)
end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)
print(f"Total ground state energy = {result.total_energies[0]:.4f}")
print(f"Total number of iterations = {result.raw_result.num_iterations}")

Code execution time [sec]: 24.282981157302856
Total ground state energy = -1.1102
Total number of iterations = 2


## now setup AdaptVQE with noisy backend and zne (qiskit)

In [15]:
from qiskit_ibm_runtime.fake_provider import FakeVigo
device_backend = FakeVigo()

/tmp/ipykernel_12485/2647586769.py:2: DeprecationWarning: V1 fake backends are deprecated as of qiskit-ibm-runtime 0.24 and will be removed no sooner than 3 months after the release date. Please use V2 fake backends instead.
  device_backend = FakeVigo()


In [16]:
from qiskit_ibm_runtime import Estimator, Options
  
options = Options()
 
options.resilience_level = 2

estimator = Estimator(device_backend, options=options)

/tmp/ipykernel_12485/486542272.py:7: DeprecationWarning: The Sampler and Estimator V1 primitives have been deprecated as of qiskit-ibm-runtime 0.23.0 and will be removed no sooner than 3 months after the release date. Please use the V2 Primitives. See the `V2 migration guide <https://docs.quantum.ibm.com/api/migration-guides/v2-primitives>`_. for more details
  estimator = Estimator(device_backend, options=options)


In [17]:
# define vqe
noisy_zne_vqe = VQE(estimator, ansatz, SLSQP())
noisy_zne_vqe.initial_point = np.zeros(ansatz.num_parameters)

In [18]:
noisy_zne_adapt_vqe = AdaptVQE(noisy_zne_vqe)
noisy_zne_adapt_vqe.supports_aux_operators = lambda: True  # temporary fix

## Now get some results AdaptVQE noisy and zne (qiskit)

In [19]:
solver = GroundStateEigensolver(mapper, noisy_zne_adapt_vqe)

In [20]:
start = time.time()
result = solver.solve(problem)
end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)
print(f"Total ground state energy = {result.total_energies[0]:.4f}")
print(f"Total number of iterations = {result.raw_result.num_iterations}")

Code execution time [sec]: 23.370811462402344
Total ground state energy = -1.1225
Total number of iterations = 2
